# LAST.FM

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import requests, time, re, html


In [4]:
my_API_KEY = "469717cb3f730cb084a39353af9ca7ab"
SHARED_SECRET = "578426e9abd4555869a1ffa181572b27"

**Extracción de Detalles de Canciones/Álbumes con la API de last.fm.**

In [ ]:
def get_artist_info(artist_name): # Generamos una función para obtener de la API la información de los artistas seleccionados en la extracción de la API de Spotipy
    API_KEY = my_API_KEY 
    BASE_URL = "https://ws.audioscrobbler.com/2.0/" # Conexión a la API de LastFM

    params = {
        "method": "artist.getinfo",
        "artist": artist_name,
        "api_key": API_KEY,
        "format": "json",
        "autocorrect": 1,
        "lang": "en"
    }                                               # Parametros dados a la API para la extración en el endpoint "artist.getinfo"

    response = requests.get(BASE_URL, params=params) 
    data = response.json()   # Acceso seguro a los datos

    # Validar si hay error
    if "error" in data:
        print(f"❌ Error: {data['message']}")
        return None

    artist = data["artist"]
    bio = artist["bio"]["summary"]
    listeners = artist["stats"]["listeners"]
    playcount = artist["stats"]["playcount"]
    similar_artists = [a["name"] for a in artist["similar"]["artist"]]
    similar_artist = similar_artists[0] if similar_artists else None
     # Información de cada canción según los parametros explicitos

    return {
        "artist_name": artist["name"],
        "biography": bio,
        "listeners": listeners,
        "playcount": playcount,
        "similar_artist": similar_artist
    } # Formato para mostrar la información

In [5]:
df_spotify = pd.read_csv("canciones_spotify.csv")  # Transformación en DataFrame
df_spotify.head()

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,1WHNqqRWhJVZIdCScFKtl5,Washington on Your Side,Leslie Odom Jr.,2015,soundtrack,album,2015-09-25,67
1,1llkMtLL4vbe0pr3NV2ckg,Let Me Hear,"Fear, and Loathing in Las Vegas",2015,soundtrack,album,2015-09-30,58
2,39IOkz6LpC1qc5Wnt0T07r,Dance of the Druids (feat. Raya Yarbrough),Bear McCreary,2015,soundtrack,album,2015-02-10,53
3,5L9MJsGqzTRD09rSzHkCDy,Home,Toby Fox,2015,soundtrack,album,2015-09-15,62
4,0LpHC9mhPAQC98IjXZIrif,The Election of 1800,Daveed Diggs,2015,soundtrack,album,2015-09-25,66


In [6]:
df_spotify['artist_name'].tolist() # Extración de lista de artistas del DataFrame superior

['Leslie Odom Jr.',
 'Fear, and Loathing in Las Vegas',
 'Bear McCreary',
 'Toby Fox',
 'Daveed Diggs',
 'Okieriete Onaodowan',
 'Aimer',
 'Toby Fox',
 'Phillipa Soo',
 'Toby Fox',
 'Phillipa Soo',
 'Original Broadway Cast of Hamilton',
 'Lin-Manuel Miranda',
 'Leslie Odom Jr.',
 'Lin-Manuel Miranda',
 'Toby Fox',
 'Original Broadway Cast of Hamilton',
 'Lin-Manuel Miranda',
 'Michael Giacchino',
 'Christopher Jackson',
 'Toby Fox',
 'Renée Elise Goldsberry',
 'Jasmine Cephas-Jones',
 'Original Broadway Cast of Hamilton',
 'Thayne Jasperson',
 'Leslie Odom Jr.',
 'Leslie Odom Jr.',
 'Lin-Manuel Miranda',
 'Leslie Odom Jr.',
 'Leslie Odom Jr.',
 'Toby Fox',
 'Leslie Odom Jr.',
 'Goose house',
 'Jonathan Groff',
 'Toby Fox',
 'Daveed Diggs',
 'Phillipa Soo',
 'Leslie Odom Jr.',
 'Jonathan Groff',
 'Toby Fox',
 'Phillipa Soo',
 'Lin-Manuel Miranda',
 'Toby Fox',
 'Lin-Manuel Miranda',
 'KANA-BOON',
 'Leslie Odom Jr.',
 'John Williams',
 'Bear McCreary',
 'Renée Elise Goldsberry',
 'Toby F

In [7]:
len(df_spotify['artist_name'].tolist()) # Número de registros de la extración de datos de la API Spotipy

3000

In [8]:
unique_artists_spotify = df_spotify['artist_name'].unique().tolist() # Generar valores únicos de artistas de la extración de datos de la API Spotipy

In [9]:
len(unique_artists_spotify) # Número de registros unicos de artistas de la extración de datos de la API Spotipy

1198

In [10]:
lista_artist_lastfm_NN = [] # Variable para almacenar toda la información de todos los artistas que no sean nulos.

for artista in unique_artists_spotify:
    info = get_artist_info(artista)
    if info is not None:  # solo agrega si tiene datos
        lista_artist_lastfm_NN.append(info)

❌ Error: The artist you supplied could not be found
❌ Error: The artist you supplied could not be found


In [11]:
len(lista_artist_lastfm_NN) # Número de registros unicos de artistas de la extración de datos de la API LastFM

1196

In [12]:
lista_artist_lastfm_NN

[{'artist_name': 'Leslie Odom Jr.',
  'biography': 'Leslie Odom, Jr (born August 6, 1981) is an American actor and singer. He has performed on Broadway and in television and film, as well as in other theatrical productions. \nHe is best known for the roles of Aaron Burr in the 2015 musical Hamilton and Sam Strickland in the 2013 musical television series Smash.\n <a href="https://www.last.fm/music/Leslie+Odom+Jr.">Read more on Last.fm</a>',
  'listeners': '782639',
  'playcount': '33453953',
  'similar_artists': 'Christopher Jackson'},
 {'artist_name': 'Fear, and Loathing in Las Vegas',
  'biography': 'Fear, and Loathing in Las Vegas is a electronicore band formed in the summer of 2008 in Kobe, Hyōgo Prefecture, Japan. The band was formed by members from Ending for a Start and Blank Time.\n\nThe band mainly plays heavy music with both high and low shouts, but unlike typical metal sounds, they have electronic beats and autotuned singing. The band spent much time searching for a clean vo

In [ ]:
## nuevo codigo de limpieza de Biography
API_KEY = my_API_KEY 

def get_bio_lastfm(artist):
    """Pide la biografía del artista desde la API."""
    url = "https://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "artist.getInfo",
        "artist": artist,
        "api_key": API_KEY,
        "format": "json",
        "autocorrect": 1,
        "lang": "en"
    }
    r = requests.get(url, params=params, timeout=15)
    if r.status_code != 200:
        print(f"❌ Error HTTP {r.status_code} para {artist}")
        return None, None
    
    data = r.json()
    bio = data.get("artist", {}).get("bio", {})
    summary = bio.get("summary",{})

    return summary


def clean_html_links(text):
    if not isinstance(text, str):
        return text
    
    # Reemplaza <a href="URL">texto</a> → URL
    text = re.sub(r'<a href="(https?://[^"]+)".*?>.*?</a>', r'\1', text)

    # Quita cualquier otra etiqueta HTML que quede
    text = re.sub(r'<.*?>', '', text)

    return text.strip()


df = pd.read_csv("artistas_lastfm_test_36.csv")

for i, artist in enumerate(df["artist_name"], start=1):
    summary = get_bio_lastfm(artist)        
    summary = clean_html_links(summary)     #aquí limpiamos el codigo HTML
    df.loc[i-1, "biography"] = summary     #aquí reemplazamos lo que está en la Bio por la info del HTML
    print(f"{i}. ✅ {artist}")
    time.sleep(0.25)

df

1. ✅ Leslie Odom Jr.
2. ✅ Fear, and Loathing in Las Vegas
3. ✅ Bear McCreary
4. ✅ Toby Fox
5. ✅ Daveed Diggs
6. ✅ Okieriete Onaodowan
7. ✅ Aimer
8. ✅ Phillipa Soo
9. ✅ Ivo Pattiera
10. ✅ krunoslav kićo slabinac
11. ✅ Группа 101
12. ✅ Tereza Kesovija
13. ✅ Badal Paul
14. ✅ Urbanus
15. ✅ Prajoth D'sa
16. ✅ Đani Maršan
17. ✅ HARI RONČEVIĆ
18. ✅ Martin Garrix
19. ✅ The Chemical Brothers
20. ✅ Cartoon
21. ✅ Dimitri Vegas & Like Mike
22. ✅ Cascada
23. ✅ Alesso
24. ✅ Caravan Palace
25. ✅ Claptone
26. ✅ David Guetta
27. ✅ Wande Coal
28. ✅ Djodje
29. ✅ Wizkid
30. ✅ Davido
31. ✅ Boy Teddy
32. ✅ C4 Pedro
33. ✅ Tekno
34. ✅ Dele Sosimi
35. ✅ Sarkodie
36. ✅ Ruff-N-Smooth


,artist_name,biography,listeners,playcount,similar_artists
0,Leslie Odom Jr.,"Leslie Odom, Jr (born August 6, 1981) is an Am...",780378,33296954,"['Christopher Jackson', 'Phillipa Soo', 'Renée..."
1,"Fear, and Loathing in Las Vegas","Fear, and Loathing in Las Vegas is a electroni...",213493,7951958,"['a crowd of rebellion', 'MY FIRST STORY', 'co..."
2,Bear McCreary,"Bear McCreary (born February 17, 1979) is an A...",520843,20700177,"['Richard Gibbs', 'Lorne Balfe', 'Howard Shore..."
3,Toby Fox,Toby Fox is an American developer and composer...,1106419,133289863,"['Lena Raine', 'Omori', 'Laura Shigihara', 'Ni..."
4,Daveed Diggs,"Daveed Daniele Diggs (born January 24, 1982 in...",395029,4603573,"['Christopher Jackson', 'Phillipa Soo', 'Lesli..."
5,Okieriete Onaodowan,"Okieriete ""Oak"" Onaodowan ( born August 16, 19...",226860,1355737,"['Thayne Jasperson', 'Christopher Jackson', 'L..."
6,Aimer,There are at least 4 artists who go by Aimer:\...,444920,23077109,"['ReoNa', 'Uru', 'Milet', 'SawanoHiroyuki[nZk]..."
7,Phillipa Soo,"Phillipa Soo (born May 31, 1990) is an America...",545794,14137529,"['Christopher Jackson', 'Leslie Odom Jr.', 'Re..."
8,Ivo Pattiera,https://www.last.fm/music/Ivo+Pattiera,448,2168,"['Klapa Maslina', 'Klapa Šibenik', 'Tomislav I..."
9,krunoslav kićo slabinac,Born in Osijek on 28th of March 1948. He start...,1008,8476,"['Zlatni Dukati', 'Gazde', 'Slavonske Lole', '..."


In [13]:
df_artist_lastfm = pd.DataFrame(lista_artist_lastfm_NN) # Transformación en DataFrame

In [14]:
df_artist_lastfm

,artist_name,biography,listeners,playcount,similar_artists
0,Leslie Odom Jr.,"Leslie Odom, Jr (born August 6, 1981) is an Am...",782639,33453953,Christopher Jackson
1,"Fear, and Loathing in Las Vegas","Fear, and Loathing in Las Vegas is a electroni...",213875,7961322,a crowd of rebellion
2,Bear McCreary,"Bear McCreary (born February 17, 1979) is an A...",521632,20716649,Richard Gibbs
3,Toby Fox,Toby Fox is an American developer and composer...,1108887,133927205,Lena Raine
4,Daveed Diggs,"Daveed Daniele Diggs (born January 24, 1982 in...",395944,4622583,Christopher Jackson
...,...,...,...,...,...
1191,Chop Daily,"<a href=""https://www.last.fm/music/Chop+Daily...",75679,591905,Keys the Prince
1192,Lola Jane,"<a href=""https://www.last.fm/music/Lola+Jane""...",5728,50640,Archie & Sizzle
1193,Praiz,"Praise Ugbede Adejo (born 8 March 1984), bette...",19967,117809,illBliss
1194,Duda,There are two groups using the name Duda one f...,6626,69608,Peled


In [14]:
df_artist_lastfm.to_csv("artistas_lastfm_en.csv", index=False) # Extración de datos a CSV